In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys; sys.path.insert(0,'../')
import pandas as pd
import tqdm
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import socceraction.atomic.vaep as vaep

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
## Configure file and folder names
datafolder = "../data"
spadl_h5 = os.path.join(datafolder,"spadl-statsbomb.h5")
predictions_h5 = os.path.join(datafolder,"predictions.h5")

In [3]:
games = pd.read_hdf(spadl_h5,"games")
games = games[games.competition_name == "FIFA World Cup"]
print("nb of games:", len(games))

nb of games: 64


In [5]:
players = pd.read_hdf(spadl_h5,"players")
teams = pd.read_hdf(spadl_h5,"teams")
actiontypes = pd.read_hdf(spadl_h5, "atomic_actiontypes")
bodyparts = pd.read_hdf(spadl_h5, "bodyparts")

A = []
for game in tqdm.tqdm(list(games.itertuples())):
    actions = pd.read_hdf(spadl_h5,f"atomic_actions/game_{game.game_id}")
    actions = (
        actions.merge(actiontypes,how="left")
        .merge(bodyparts,how="left")
        .merge(players,how="left")
        .merge(teams,how="left",)
        .sort_values(["game_id","period_id","action_id"])
        .reset_index(drop=True)
    )
    preds = pd.read_hdf(predictions_h5,f"game_{game.game_id}")
    values = vaep.value(actions,preds.scores,preds.concedes)
    A.append(pd.concat([actions,preds,values],axis=1))
A = pd.concat(A)#.sort_values(["game_id","period_id", "time_seconds", "timestamp"]).reset_index(drop=True)
A.columns

100%|██████████| 64/64 [00:03<00:00, 17.32it/s]


Index(['game_id', 'period_id', 'action_id', 'time_seconds', 'timestamp',
       'team_id', 'player_id', 'x', 'y', 'dx', 'dy', 'type_id', 'bodypart_id',
       'type_name', 'bodypart_name', 'country_id', 'country_name', 'extra',
       'jersey_number', 'player_name', 'player_nickname', 'team_name',
       'scores', 'concedes', 'offensive_value', 'defensive_value',
       'vaep_value'],
      dtype='object')

### Most valuable players in the world cup

In [6]:
A["player"] = A[["player_nickname","player_name"]].apply(lambda x: x[0] if x[0] else x[1],axis=1)

A["count"] = 1

A_ = A[A.type_name != "shot_penalty"] # ignore penalties
players = (
    A_[["player_id","team_name","player","vaep_value","count"]]
    .groupby(["player_id","team_name","player"])
    .sum()
    .reset_index()
)
players = players.sort_values("vaep_value",ascending=False)
players

,player_id,team_name,player,vaep_value,count
36,3244.0,England,John Stones,2.671270,1442
235,5457.0,Croatia,Ivan Strinić,2.463103,830
239,5461.0,Croatia,Šime Vrsaljko,2.411546,1030
174,5209.0,Portugal,Raphaël Guerreiro,2.326118,702
342,5564.0,Mexico,Carlos Salcedo,2.222801,678
265,5487.0,France,Antoine Griezmann,2.208839,800
400,5623.0,South Korea,Lee Jae-Sung,2.203069,375
140,5174.0,Russia,Sergei Ignashevich,2.129422,497
423,5646.0,Panama,Aníbal Godoy,2.071128,341
468,5691.0,Colombia,Johan Mojica,1.934616,632


### Normalize for minutes played

In [ ]:
pg = pd.read_hdf(spadl_h5,"player_games")
pg = pg[pg.game_id.isin(games.game_id)]
mp = pg[["player_id","minutes_played"]].groupby("player_id").sum().reset_index()
stats = players.merge(mp)
stats = stats[stats.minutes_played > 150]
stats["vaep_rating"] = stats.vaep_value * 90 / stats.minutes_played
stats.sort_values("vaep_value",ascending=False)